In [3]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

cap = cv.VideoCapture('VID_20251210_135648.mp4')

In [4]:
CHESSBOARD = (9, 6)
SQUARE_SIZE = 1.0

objp = np.zeros((CHESSBOARD[0] * CHESSBOARD[1], 3), dtype=np.float32)
objp[:, :2] = np.mgrid[0:CHESSBOARD[0], 0:CHESSBOARD[1]].T.reshape(-1, 2)

objpoints = []
imgpoints = []

In [5]:
# Set up termination criteria for corner subpixel refinement
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


frame_idx = 0
while True:
    # take every 5th frame
    for _ in range(4):
        _ = cap.read()
    ret, frame = cap.read()
    if not ret:
        break   # end of video
    frame_idx += 5

    img = frame
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chessboard's interior corners
    ret, corners = cv.findChessboardCorners(gray, patternSize=CHESSBOARD, corners=None)

    if ret:
        # If corners were successfully found
        # Add object points
        objpoints.append(objp)  # same for every image, since we know the geometry

        # Refine the detected image corners, and add them
        corners2 = cv.cornerSubPix(
            image=gray,
            corners=corners,
            winSize=(11, 11),
            zeroZone=(-1, -1),
            criteria=criteria,
        )
        imgpoints.append(corners2)
    else:
        print(f"Failed to detect chessboard in frame '{frame_idx}'.")
        objpoints.append(None)
        imgpoints.append(None)
    
cap.release()

Failed to detect chessboard in frame '135'.
Failed to detect chessboard in frame '140'.
Failed to detect chessboard in frame '310'.
Failed to detect chessboard in frame '315'.
Failed to detect chessboard in frame '320'.
Failed to detect chessboard in frame '345'.
Failed to detect chessboard in frame '350'.
Failed to detect chessboard in frame '355'.
Failed to detect chessboard in frame '360'.
Failed to detect chessboard in frame '390'.
Failed to detect chessboard in frame '405'.
Failed to detect chessboard in frame '410'.


In [7]:
objpoints = [objp for objp in objpoints if objp is not None]
imgpoints = [imgp for imgp in imgpoints if imgp is not None]


# Grab a single frame so that we can get image size (and use it for vis)
def get_single_frame():
    cap = cv.VideoCapture('VID_20251210_135648.mp4')

    for _ in range(20):
        ret, frame = cap.read()
    
    cap.release()
    return cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

frame = get_single_frame()


ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    objectPoints=objpoints,
    imagePoints=imgpoints,
    imageSize=frame.shape[::-1],
    cameraMatrix=None,  # We want to fit this!
    distCoeffs=None,    # We want to fit this!
)
# ret - return code
# mtx - camera intrinsics matrix
# dist - distortion coefficients
# rvecs - (extrinsic) rotation vectors
# tvecs - (extrinsic) translation vectors

In [11]:
mean_error = 0.0

for i in range(len(objpoints)):
    # Use the determined parameters to re-project the object points
    imgpoints_proj, _ = cv.projectPoints(
        objectPoints=objpoints[i],
        rvec=rvecs[i],
        tvec=tvecs[i],
        cameraMatrix=mtx,
        distCoeffs=dist,
    )
    # Calculate the norm
    error = cv.norm(imgpoints[i], imgpoints_proj, cv.NORM_L2) / len(imgpoints_proj)
    mean_error += error

mean_error /= len(objpoints)

print(f"Mean reprojection error: {mean_error:0.2f}")

Mean reprojection error: 0.12


In [10]:
# save the fitted parameters
np.savez(
    "phonecam_calib.npz",
    mtx=mtx,
    dist=dist,
    rvecs=rvecs,
    tvecs=tvecs
)